In [1]:
import cv2
import numpy as np
import os

# --- Configuration ---
image_folder = "C:/Users/BCI-Lab/Downloads/teamA_dataset/_out_dataset/good_data"
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff"))]

print(f"Loaded {len(image_files)} images.")
if len(image_files) == 0:
    print("No images found. Check your folder path or extensions.")
    exit()

current_index = 0

cv2.namedWindow("Tuning", cv2.WINDOW_NORMAL)
# Increased window size to accommodate 6 images. You might need to maximize it.
cv2.resizeWindow("Tuning", 1920, 1080) 

# Create sliders for L*a*b* color space
cv2.createTrackbar("Lower L", "Tuning", 0, 255, lambda x: None)
cv2.createTrackbar("Upper L", "Tuning", 255, 255, lambda x: None)
cv2.createTrackbar("Lower A", "Tuning", 140, 255, lambda x: None)
cv2.createTrackbar("Upper A", "Tuning", 255, 255, lambda x: None)
cv2.createTrackbar("Lower B", "Tuning", 140, 255, lambda x: None)
cv2.createTrackbar("Upper B", "Tuning", 255, 255, lambda x: None)

# Morphological Kernel for Color Mask
cv2.createTrackbar("Color Morph Kernel", "Tuning", 5, 50, lambda x: None) 

# Morphological Kernel for Edge Mask
cv2.createTrackbar("Edge Morph Kernel", "Tuning", 3, 50, lambda x: None) 

# Canny Edge Detector Thresholds
cv2.createTrackbar("Canny Thresh1", "Tuning", 50, 255, lambda x: None)
cv2.createTrackbar("Canny Thresh2", "Tuning", 150, 255, lambda x: None)

# NEW: Hough Line Transform Parameters
cv2.createTrackbar("Hough Threshold", "Tuning", 50, 200, lambda x: None) # Accumulator threshold
cv2.createTrackbar("Hough Min Length", "Tuning", 50, 500, lambda x: None) # Minimum line length
cv2.createTrackbar("Hough Max Gap", "Tuning", 10, 200, lambda x: None) # Maximum gap to connect segments

cv2.createTrackbar("Crop % from Top", "Tuning", 75, 100, lambda x: None)


# --- Main Loop ---
while True:
    filename = image_files[current_index]
    print(f"Showing {filename} at index {current_index}")

    filepath = os.path.join(image_folder, filename)
    image = cv2.imread(filepath)
    if image is None:
        print(f"❌ Cannot read {filename}. Skipping.")
        current_index = (current_index + 1) % len(image_files)
        continue

    # Get trackbar values
    ll = cv2.getTrackbarPos("Lower L", "Tuning")
    ul = cv2.getTrackbarPos("Upper L", "Tuning")
    la = cv2.getTrackbarPos("Lower A", "Tuning")
    ua = cv2.getTrackbarPos("Upper A", "Tuning")
    lb = cv2.getTrackbarPos("Lower B", "Tuning")
    ub = cv2.getTrackbarPos("Upper B", "Tuning")
    
    color_morph_kernel_size = max(1, cv2.getTrackbarPos("Color Morph Kernel", "Tuning")) 
    edge_morph_kernel_size = max(1, cv2.getTrackbarPos("Edge Morph Kernel", "Tuning")) 
    
    canny_thresh1 = cv2.getTrackbarPos("Canny Thresh1", "Tuning")
    canny_thresh2 = cv2.getTrackbarPos("Canny Thresh2", "Tuning")

    # NEW: Get Hough parameters
    hough_threshold = cv2.getTrackbarPos("Hough Threshold", "Tuning")
    hough_min_length = cv2.getTrackbarPos("Hough Min Length", "Tuning")
    hough_max_gap = cv2.getTrackbarPos("Hough Max Gap", "Tuning")

    crop_percent = cv2.getTrackbarPos("Crop % from Top", "Tuning")

    # --- 1. Crop from top ---
    crop_y = int(image.shape[0] * crop_percent / 100)
    if crop_y >= image.shape[0]:
        crop_y = image.shape[0] - 1
    cropped_image = image[crop_y:, :].copy()

    # --- 2. Color Masking (L*a*b*) ---
    lab_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_cropped)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_eq = clahe.apply(l_channel)
    lab_eq = cv2.merge([l_eq, a_channel, b_channel])
    blurred_lab_eq = cv2.medianBlur(lab_eq, 5)

    lower_orange_lab = np.array([ll, la, lb])
    upper_orange_lab = np.array([ul, ua, ub])
    color_mask = cv2.inRange(blurred_lab_eq, lower_orange_lab, upper_orange_lab)

    # --- 3. Morphological Operations on Color Mask ---
    color_morph_kernel = np.ones((color_morph_kernel_size, color_morph_kernel_size), np.uint8)
    color_mask_morphed = cv2.morphologyEx(color_mask, cv2.MORPH_OPEN, color_morph_kernel, iterations=1)
    color_mask_morphed = cv2.dilate(color_mask_morphed, color_morph_kernel, iterations=1) 
    color_mask_morphed = cv2.morphologyEx(color_mask_morphed, cv2.MORPH_CLOSE, color_morph_kernel, iterations=1)
    
    # --- 4. Edge Detection (Canny) ---
    gray_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    blurred_gray = cv2.GaussianBlur(gray_cropped, (5, 5), 0)
    edge_mask = cv2.Canny(blurred_gray, canny_thresh1, canny_thresh2)

    # Morphological Operations on Edge Mask to connect broken edges
    edge_morph_kernel = np.ones((edge_morph_kernel_size, edge_morph_kernel_size), np.uint8)
    edge_mask_morphed = cv2.morphologyEx(edge_mask, cv2.MORPH_CLOSE, edge_morph_kernel, iterations=1)

    # --- 5. Combine Color Mask and Morphed Edge Mask ---
    final_mask = cv2.bitwise_and(color_mask_morphed, edge_mask_morphed)

    # --- 6. Line Detection with Hough Transform ---
    # Apply HoughLinesP to the final_mask (which is binary and has cleaned edges)
    # rho=1 (1 pixel resolution), theta=np.pi/180 (1 degree resolution)
    # Using Hough Min Length and Max Gap from trackbars
    lines = cv2.HoughLinesP(final_mask, 1, np.pi / 180, 
                            hough_threshold, 
                            minLineLength=hough_min_length, 
                            maxLineGap=hough_max_gap)
    
    # --- 7. Prepare Preview and Determine Line Position ---
    preview = image.copy()

    # Draw crop zone
    cv2.rectangle(preview, (0, crop_y), (image.shape[1], image.shape[0]), (255, 0, 255), 2)
    
    # Draw center line for visual reference
    img_width = image.shape[1]
    center_x_img = img_width // 2
    cv2.line(preview, (center_x_img, crop_y), (center_x_img, image.shape[0]), (0, 255, 255), 2)

    line_position_label = "No Line Detected (2)"
    cx = -1 # Default cx if no line found
    
    if lines is not None:
        all_line_midpoints_x = []
        for line in lines:
            x1, y1, x2, y2 = line[0]
            # Draw detected lines on the preview image (adjust y-coordinates for original image)
            cv2.line(preview, (x1, y1 + crop_y), (x2, y2 + crop_y), (0, 255, 0), 2)
            # Calculate midpoint x for each line segment and store it
            all_line_midpoints_x.append((x1 + x2) // 2)
        
        if all_line_midpoints_x:
            cx = int(np.mean(all_line_midpoints_x))
            # Choose a representative y for the centroid for drawing, e.g., middle of the crop
            cy = crop_y + (image.shape[0] - crop_y) // 2 

            cv2.circle(preview, (cx, cy), 6, (0, 0, 255), -1) # Red circle for centroid
            cv2.putText(preview, f"cx: {cx}", (cx + 10, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

            tolerance = img_width * 0.1 
            
            if abs(cx - center_x_img) < tolerance:
                line_position_label = "Line in Center (0)"
            elif cx < center_x_img - tolerance:
                line_position_label = "Line on Left (-1)"
            else:
                line_position_label = "Line on Right (1)"
    
    cv2.putText(preview, line_position_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

    # --- Display images with consistent, larger size ---
    display_height = 300 # Reduced height slightly to make more room for sliders
    
    # Calculate aspect ratio for original image and use it for all resizes
    original_h, original_w, _ = image.shape
    aspect_ratio = original_w / original_h
    display_width = int(display_height * aspect_ratio)

    preview_resized = cv2.resize(preview, (display_width, display_height))
    cropped_image_resized = cv2.resize(cropped_image, (display_width, display_height))
    color_mask_resized = cv2.resize(cv2.cvtColor(color_mask, cv2.COLOR_GRAY2BGR), (display_width, display_height))
    morphed_mask_resized = cv2.resize(cv2.cvtColor(color_mask_morphed, cv2.COLOR_GRAY2BGR), (display_width, display_height))
    edge_mask_morphed_resized = cv2.resize(cv2.cvtColor(edge_mask_morphed, cv2.COLOR_GRAY2BGR), (display_width, display_height))
    final_mask_resized = cv2.resize(cv2.cvtColor(final_mask, cv2.COLOR_GRAY2BGR), (display_width, display_height))

    # Stack for display: Annotated Preview | Cropped Original | Initial Color Mask | Morphed Color Mask | Morphed Edge Mask | Final Mask
    combined_display = np.hstack((
        preview_resized,
        cropped_image_resized,
        color_mask_resized,
        morphed_mask_resized,
        edge_mask_morphed_resized, 
        final_mask_resized
    ))

    cv2.imshow("Tuning", combined_display)

    key = cv2.waitKey(30) & 0xFF
    if key == ord("q"):
        break
    elif key in [ord("d"), 83]:  # next image (D or Right Arrow)
        current_index = (current_index + 1) % len(image_files)
    elif key in [ord("a"), 81]:  # previous image (A or Left Arrow)
        current_index = (current_index - 1 + len(image_files)) % len(image_files) 
    elif key == ord("s"):
        print(f"\n✅ Saved params for {filename}:")
        print(f"lower_orange_lab = np.array([{ll}, {la}, {lb}])")
        print(f"upper_orange_lab = np.array([{ul}, {ua}, {ub}])")
        print(f"color_morph_kernel_size = {color_morph_kernel_size}")
        print(f"edge_morph_kernel_size = {edge_morph_kernel_size}") 
        print(f"canny_thresh1 = {canny_thresh1}")
        print(f"canny_thresh2 = {canny_thresh2}")
        print(f"hough_threshold = {hough_threshold}")
        print(f"hough_min_length = {hough_min_length}")
        print(f"hough_max_gap = {hough_max_gap}")
        print(f"crop_percent = {crop_percent}")
        print(f"Predicted position for this image: {line_position_label} (cx={cx})\n")

cv2.destroyAllWindows()


Loaded 1291 images.
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png at index 0
Showing 00420478.png

KeyboardInterrupt: 